In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torchvision.io import read_image, ImageReadMode
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    break
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#Pre-processing
NORMAL = 0
PNEUMONIA = 1
# transforms = v2.Compose([
#                  v2.ToImage(),  # Convert to tensor, only needed if you had a PIL image
#                  v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
#                  # v2.RandomResizedCrop(size=(256, 256), antialias=True),  # Or Resize(antialias=True)
#                  v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
#                  v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#                  ])
transforms = v2.Compose([
#                     v2.Resize((512, 512))
                    v2.Resize((128, 128)),
                    v2.Grayscale(num_output_channels=3),
                    v2.ToTensor()
#                     v2.Resize((32, 32))
#                     v2.RandomResizedCrop((32, 12832
                    ])
path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'
def make_dataframe(dataset_name):
    normal_path = f'/kaggle/input/chest-xray-pneumonia/chest_xray/{dataset_name}/NORMAL'
    pneumonia_path = f'/kaggle/input/chest-xray-pneumonia/chest_xray/{dataset_name}/PNEUMONIA'
    X = []
    Y = []
    with os.scandir(normal_path) as it:
        for entry in it:
            X.append(entry.path)
            Y.append(NORMAL)

    with os.scandir(pneumonia_path) as it:
        for entry in it:
            X.append(entry.path)
            Y.append(PNEUMONIA)

    data = {'image': X, 'label': Y}
    df = pd.DataFrame(data)
    return df

test_df = make_dataframe('test')
train_df = make_dataframe('train')
val_df = make_dataframe('val')

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, target_transform=None):
        self.img_labels = dataframe['label']
        self.img_paths = dataframe['image']
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_paths.iloc[idx]
        image = read_image(img_path, ImageReadMode.RGB).to('cuda')
        label = self.img_labels.iloc[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [5]:
def show_image(tensor):
    arr_ = np.squeeze(tensor)
    plt.imshow(arr_, cmap='gray')
    plt.show()

In [6]:
test_dataset = CustomImageDataset(test_df, transform=transforms)
train_dataset = CustomImageDataset(train_df, transform=transforms)
val_dataset = CustomImageDataset(val_df, transform=transforms)
# tensor = train_dataset.__getitem__(20)[0]
# show_image(tensor.to('cpu'))
# train_dataset.__getitem__(39)[0].shape

In [7]:
batch_size = 16

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, batch_size):
        super().__init__()
#         self.conv1 = nn.Conv2d(1, 16, 258, stride=2)
#         self.conv2 = nn.Conv2d(16, 32, 2, stride=2)
#         self.pool1 = nn.AvgPool2d(5, stride=1)
#         self.conv3 = nn.Conv2d(32, 64, 29)
#         self.pool2 = nn.MaxPool2d(18, 2)
#         self.fc1 = nn.Linear(64 * 8 * 8 * batch_size, 1024* batch_size)
#         self.fc2 = nn.Linear(1024* batch_size, 128* batch_size)
#         self.fc3 = nn.Linear(128* batch_size, 16* batch_size)
#         self.fc4 = nn.Linear(16* batch_size, 1* batch_size)
#         self.conv1 = nn.Conv2d(1, 4, 65)
#         self.batchnorm1 = nn.BatchNorm2d(4)
#         self.conv2 = nn.Conv2d(4, 8, 33)
#         self.batchnorm2 = nn.BatchNorm2d(8)
#         self.pool1 = nn.AvgPool2d(2, stride=1)
#         self.conv3 = nn.Conv2d(8, 16, 15)
#         self.batchnorm3 = nn.BatchNorm2d(16)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(16 * 8 * 8 * batch_size, 128* batch_size)
#         self.fc2 = nn.Linear(128* batch_size, 64* batch_size)
#         self.fc3 = nn.Linear(64* batch_size, 16* batch_size)
#         self.fc4 = nn.Linear(16* batch_size, 1* batch_size)
        self.conv1 = nn.Conv2d(1, 8, 2, stride=2)
        self.conv2 = nn.Conv2d(8, 16, 2, stride=1)
        self.pool1 = nn.AvgPool2d(2, stride=1)
        self.conv3 = nn.Conv2d(16, 32, 2)
        self.pool2 = nn.MaxPool2d(6, stride=1)
        self.fc1 = nn.Linear(32 * 8 * 8 * batch_size, 512 * batch_size)
        self.fc2 = nn.Linear(512* batch_size, 64* batch_size)
        self.fc3 = nn.Linear(64* batch_size, 8* batch_size)
        self.fc4 = nn.Linear(8* batch_size, 1*batch_size)

    def forward(self, x):
#         x = self.pool1(F.relu(self.batchnorm2(self.conv2(F.relu(self.batchnorm1(self.conv1(x)))))))
#         x = self.pool2(F.relu(self.batchnorm3(self.conv3(x))))
#         x = torch.flatten(x)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
# #         x = F.sigmoid(self.fc4(x))
#         x = self.fc4(x)
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv3(x)))
#         print(x.shape)
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
#         x = F.sigmoid(self.fc4(x))
        x = self.fc4(x)
        return x


net = Net(batch_size)
net.to('cuda')

Net(
  (conv1): Conv2d(1, 8, kernel_size=(2, 2), stride=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=1, padding=0)
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=6, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=8192, bias=True)
  (fc2): Linear(in_features=8192, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=16, bias=True)
)

In [9]:
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

def CNN_Model(pretrained=True):
    model = models.densenet121(pretrained=pretrained) # Returns Defined Densenet model with weights trained on ImageNet
    num_ftrs = model.classifier.in_features # Get the number of features output from CNN layer
    model.classifier = nn.Linear(num_ftrs, 1) # Overwrites the Classifier layer with custom defined layer for transfer learning
    model = model.to('cuda') # Transfer the Model to GPU if available
    return model
net = CNN_Model()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 122MB/s] 


In [10]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: Tesla T4 is available.


In [12]:
def train_model(loader):
    epochs = 1000
    running_loss = 0.0
    epoch = 0
    for data in loader:
        try:
            if epoch >= epochs:
                break
        #     if epoch % 10 == 0:
        #         print("Processed: " + str(epoch))
            optimizer.zero_grad()
            image, target = data
            # forward + backward + optimize
            inp = image.type(torch.FloatTensor).to('cuda')
            label = target.type(torch.FloatTensor).to('cuda').reshape((16,1))
            outputs = net(inp)
#             print(outputs)
        #     print(outputs.|shape)
        #     print(label.shape)
            loss = criterion(outputs, label).to('cuda')
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if epoch % 100 == 99:    # print every 100 mini-batches
                print(f'[{epoch + 1}, {epoch + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
            epoch = epoch + 1
        except Exception as e:
            print("Error: " + str(e))
            print(data[0].shape)
            break

    print('Finished Training')
    
net.train()
# for _ in range(3):
train_model(train_loader)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[100,   100] loss: 0.012
[200,   200] loss: 0.005
[300,   300] loss: 0.004
Finished Training


In [13]:
def test(model, device, loader, dataset):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.type(torch.FloatTensor).to('cuda'), target.type(torch.FloatTensor).to('cuda').reshape((16,1))
            output = model(data)
            output = (output >= 0.5).float()
            test_loss += F.binary_cross_entropy(output, target, reduction='sum').item() # sum up batch loss
#             print(output, target)
            correct += output.eq(target.view_as(output)).sum().item()

    test_loss /= len(loader.dataset)

    print('\n{} set: Accuracy: {}/{} ({:.0f}%)\n'.format(dataset,
         correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

In [14]:
test(net, 'cuda', train_loader, 'Train')


Train set: Accuracy: 5163/5216 (99%)



In [15]:
test(net, 'cuda', val_loader, 'Val')


Val set: Accuracy: 13/16 (81%)



In [16]:
test(net, 'cuda', test_loader, 'Test')


Test set: Accuracy: 508/624 (81%)

